In [ ]:
import requests
import re
from enum import Enum
from time import localtime, strftime
import os
from dotenv import load_dotenv
load_dotenv()

class ScannerAdfState(Enum):
    EMPTY = "ScannerAdfEmpty"
    LOADED = "ScannerAdfLoaded"
    JAMMED = "ScannerAdfMispick"

class ScannerState(Enum):
    IDLE = "Idle"
    PROCESSING = "Processing"
    STOPPED = "Stopped"

class JobState(Enum):
    COMPLETED = "Completed"
    PROCESSING = "Processing"
    PENDING = "Pending" # wait for ADF to start scanning
    FAILED = "Aborted"

class ScannerInfo:
    def __init__(self):
        self.target_dir = "out"
        self.ip = os.getenv('IP_ADDRESS')
        self.target_dir = os.getenv('TARGET_DIR')
        self.file_format = os.getenv('FILE_FORMAT')
        url = f"http://{self.ip}/eSCL/ScannerStatus"
        r = requests.get(url)
        self.response = r.text
        self.adf_state = re.search(r'<scan:AdfState>(.+)</scan:AdfState>', r.text).group(1)
        self.scanner_state = re.search(r'<pwg:State>(.+)</pwg:State>', r.text).group(1)
        self.last_job_id = re.search(r'<pwg:JobUuid>(.+)</pwg:JobUuid>', r.text).group(1)
        self.last_job_state = re.search(r'<pwg:JobState>(.+)</pwg:JobState>', r.text).group(1)

    def print_info(self, include_response=False):
        print(f"ADF State: {self.adf_state}")
        print(f"Scanner State: {self.scanner_state}")
        print(f"Last Job ID: {self.last_job_id}")
        print(f"Last Job State: {self.last_job_state}")
        if include_response:
            print("Full Response:")
            print(self.response)

    def is_adf_loaded(self):
        return self.adf_state == ScannerAdfState.LOADED.value
    
    def is_scanner_idle(self):
        return self.scanner_state == ScannerState.IDLE.value

    def start_glass_scan(self):
        url = f"http://{self.ip}/eSCL/ScanJobs"
        glass_payload = """
        <scan:ScanSettings xmlns:scan="http://schemas.hp.com/imaging/escl/2011/05/03"
            xmlns:copy="http://www.hp.com/schemas/imaging/con/copy/2008/07/07"
            xmlns:dd="http://www.hp.com/schemas/imaging/con/dictionaries/1.0/"
            xmlns:dd3="http://www.hp.com/schemas/imaging/con/dictionaries/2009/04/06"
            xmlns:fw="http://www.hp.com/schemas/imaging/con/firewall/2011/01/05"
            xmlns:scc="http://schemas.hp.com/imaging/escl/2011/05/03"
            xmlns:pwg="http://www.pwg.org/schemas/2010/12/sm">
            <pwg:Version>2.1</pwg:Version>
            <scan:Intent>Document</scan:Intent>
            <pwg:ScanRegions>
                <pwg:ScanRegion>
                    <pwg:Height>3507</pwg:Height>
                    <pwg:Width>2481</pwg:Width>
                    <pwg:XOffset>0</pwg:XOffset>
                    <pwg:YOffset>0</pwg:YOffset>
                </pwg:ScanRegion>
            </pwg:ScanRegions>
            <pwg:InputSource>Platen</pwg:InputSource>
            <scan:DocumentFormatExt>application/pdf</scan:DocumentFormatExt>
            <scan:XResolution>300</scan:XResolution>
            <scan:YResolution>300</scan:YResolution>
            <scan:ColorMode>RGB24</scan:ColorMode>
            <scan:CompressionFactor>35</scan:CompressionFactor>
            <scan:Brightness>600</scan:Brightness>
            <scan:Contrast>600</scan:Contrast>
        </scan:ScanSettings>"""
        requests.post(url, glass_payload)

    def start_adf_scan(self):
        url = f"http://{self.ip}/eSCL/ScanJobs"
        glass_payload = """
        <scan:ScanSettings xmlns:scan="http://schemas.hp.com/imaging/escl/2011/05/03"
            xmlns:copy="http://www.hp.com/schemas/imaging/con/copy/2008/07/07"
            xmlns:dd="http://www.hp.com/schemas/imaging/con/dictionaries/1.0/"
            xmlns:dd3="http://www.hp.com/schemas/imaging/con/dictionaries/2009/04/06"
            xmlns:fw="http://www.hp.com/schemas/imaging/con/firewall/2011/01/05"
            xmlns:scc="http://schemas.hp.com/imaging/escl/2011/05/03"
            xmlns:pwg="http://www.pwg.org/schemas/2010/12/sm">
            <pwg:Version>2.1</pwg:Version>
            <scan:Intent>Document</scan:Intent>
            <pwg:ScanRegions>
                <pwg:ScanRegion>
                    <pwg:Height>3507</pwg:Height>
                    <pwg:Width>2481</pwg:Width>
                    <pwg:XOffset>0</pwg:XOffset>
                    <pwg:YOffset>0</pwg:YOffset>
                </pwg:ScanRegion>
            </pwg:ScanRegions>
            <pwg:InputSource>Feeder</pwg:InputSource>
            <scan:DocumentFormatExt>application/pdf</scan:DocumentFormatExt>
            <scan:XResolution>300</scan:XResolution>
            <scan:YResolution>300</scan:YResolution>
            <scan:ColorMode>RGB24</scan:ColorMode>
            <scan:Duplex>false</scan:Duplex>
            <scan:CompressionFactor>15</scan:CompressionFactor>
            <scan:Brightness>1000</scan:Brightness>
            <scan:Contrast>1000</scan:Contrast>
        </scan:ScanSettings>"""
        requests.post(url, glass_payload)
    
    def start_download(self):
        url = f"http://{self.ip}/eSCL/ScanJobs/{self.last_job_id}/NextDocument"
        r = requests.get(url)
        filename = strftime(self.file_format, localtime()) + ".pdf"
        with open(f"{self.target_dir}/{filename}", mode="wb") as file:
            file.write(r.content)

info = ScannerInfo()
info.print_info()
info.start_adf_scan() # Starts a new Job -> new ScannerInfoObject needed to get the new job ID
ScannerInfo().start_download()

ADF State: ScannerAdfLoaded
Scanner State: Idle
Last Job ID: a72deb67-8463-4df1-a0d1-7240120c8510
Last Job State: Completed
